---
title: "Cosma Core: Agent Class for Composable Agentic Workflows"
author: "Charles F. Vardeman II"
description: |
  This notebook defines the `Agent` class, which provides the foundation for composable
  agentic workflows using Cosma. It includes:
  - A flexible `Agent` class for creating tool-using LLM agents.
  - Support for composable and extensible AI workflows.
  - Integration with cosette, OpenAI API, and Anthropic-recommended patterns.
format:
  html:
    toc: true
    include-in-header:
      text: |
        <script type="application/ld+json">
        {
          "@context": "https://doi.org/10.5063/schema/codemeta-2.0",
          "@type": "SoftwareSourceCode",
          "name": "Cosma Core: Agent Class",
          "description": "Defines the `Agent` class, providing the basis for composable agentic workflows in Cosma.",
          "author": {
            "@type": "Person",
            "name": "Charles F. Vardeman II",
            "affiliation": {
              "@type": "Organization",
              "name": "Laboratory for Assured AI Application Development (LA3D), Center for Research Computing, University of Notre Dame"
            }
          },
          "creator": {
            "@type": "Person",
            "name": "Charles F. Vardeman II"
          },
          "publisher": {
            "@type": "Organization",
            "name": "University of Notre Dame"
          },
          "copyrightHolder": {
            "@type": "Organization",
            "name": "University of Notre Dame"
          },
          "programmingLanguage": "Python",
          "license": "https://opensource.org/licenses/MIT",
          "keywords": ["LLM agents", "AI", "cosette", "OpenAI", "Anthropic", "Agentic workflows", "Python"],
          "codeRepository": "https://github.com/your-repository/cosma",
          "issueTracker": "https://github.com/your-repository/cosma/issues",
          "softwareVersion": "0.1.0",
          "dateCreated": "2025-02-15",
          "dateModified": "2025-02-15",
          "developmentStatus": "active",
          "operatingSystem": "Any",
          "softwareRequirements": ["Python >= 3.8"],
          "funding": {
            "@type": "Organization",
            "name": "Funding Agency Name"
          },
          "isAccessibleForFree": true,
          "mainEntity": {
            "@type": "Class",
            "name": "Agent",
            "description": "A flexible class for creating tool-using LLM agents and supporting agentic workflows."
          }
        }
        </script>
---

# core

> Core Agent functionality for building LLM-powered agents with [cosette](https://github.com/AnswerDotAI/cosette).

::: {.callout-note}
This nbdev notebook was dialog engineered in the ["Extending Cosette with more agentic functionality"](https://github.com/LA3D/cosma/blob/main/dialogs/Cosma_Refactor.ipynb). There is an example walkthrough of using the `Agent` class in the ["Example Agent"](https://github.com/LA3D/cosma/blob/main/dialogs/Cosma-walkthru.ipynb) notebook.
:::

In [ ]:
#| default_exp core


In [ ]:
#| export
from fastcore.utils import *
from fastcore.basics import patch
from dataclasses import dataclass, field
from typing import Any, Optional, List, Callable, Dict
from cosette import Chat, contents, wrap_latex, models
from IPython.display import display, Markdown



In [ ]:
#| hide
from nbdev.showdoc import *
from dotenv import load_dotenv

## Load keys for testing.
load_dotenv()

True

## Basic Tools
> Example tools for demonstrating agent functionality

Tools must be designed with clear documentation and examples for the LLM to use them effectively. Following Anthropic's guidance:
1. Use clear, descriptive parameter names
2. Include comprehensive docstrings with examples
3. Specify input formats and constraints
4. Show example usage patterns

In [ ]:
# | export
import math

def solve_math(
    expression: str  # Mathematical expression as a string (e.g. "2+2", "sqrt(16)")
) -> float:         # Numerical result of the evaluation
    """Evaluates mathematical expressions using a safe subset of Python's math operations.
    
    The tool supports these operations:
    - Basic arithmetic: +, -, *, /
    - Functions: sqrt, pow, sin, cos
    - Constants: pi
    
    Examples:
        >>> solve_math("2+2")
        4.0
        >>> solve_math("sqrt(16)")
        4.0
        >>> solve_math("sin(pi/2)")
        1.0
        
    Input Format:
        - Use standard mathematical notation
        - Write functions in lowercase: sqrt(), sin(), cos()
        - Use parentheses for function arguments: sqrt(16)
        
    Safety:
        - Only whitelisted math operations are allowed
        - No arbitrary Python code execution
    """
    namespace = {
        'sqrt': math.sqrt,
        'pow': math.pow,
        'sin': math.sin,
        'cos': math.cos,
        'pi': math.pi
    }
    return eval(expression, {"__builtins__": {}}, namespace)

In [ ]:
# | hide
# Test basic operations
test_cases = [
    ("2+2", "Basic addition"),
    ("sqrt(16)", "Square root function"),
    ("sin(pi/2)", "Trigonometric with pi constant"),
    ("pow(2,3)", "Power function"),
    ("cos(0)", "Cosine of zero")
]

for expr, desc in test_cases:
    result = solve_math(expr)
    print(f"{desc:25} | {expr:10} = {result}")

Basic addition            | 2+2        = 4
Square root function      | sqrt(16)   = 4.0
Trigonometric with pi constant | sin(pi/2)  = 1.0
Power function            | pow(2,3)   = 8.0
Cosine of zero            | cos(0)     = 1.0


## Agent Class
> Core class for building LLM-powered agents with tools and memory

The Agent class provides a high-level interface for creating LLM agents that can:
- Maintain conversation history
- Use well-documented tools effectively
- Follow specific roles and system prompts
- Manage context window automatically


In [ ]:
# | export
@dataclass
class Agent:
    """An Agent that can perform tasks using an LLM and optional tools.
    
    The Agent maintains its own conversation state and can use tools to perform
    actions. It follows Anthropic's best practices for tool usage and prompting.
    
    Args:
        role: Description of agent's role (e.g. "math tutor")
        model: LLM model to use (from cosette.models)
        tools: Optional list of callable tools with type hints and docstrings
        system: Override default system prompt
        memory_size: Number of conversation turns to retain
    
    Example:
        ```python
        # Create a math tutor agent
        math_agent = Agent(
            role="math tutor",
            model="gpt-4o",
            tools=[solve_math],
            system="You are a helpful math tutor. Show your work and verify with tools."
        )
        
        # Use the agent
        response = math_agent.run_with_tools("What is sqrt(16) + 7?")
        ```
    """
    role: str
    model: str
    tools: List[Callable] = field(default_factory=list)
    system: Optional[str] = None
    memory_size: int = 10
    
    def __post_init__(self):
        """Initialize the agent with model and system prompt."""
        if self.model not in models: 
            raise ValueError(f"Model {self.model} not in available models: {models}")
        self.chat = Chat(self.model, tools=self.tools)
        self.chat.sp = self.system or f"You are a {self.role}."


In [ ]:
# | hide
# Test basic agent creation
test_agent = Agent(
    role="test agent",
    model=models[2],  # Use available model
    tools=[solve_math]
)
print(f"Created agent with model: {test_agent.model}")
print(f"System prompt: {test_agent.chat.sp}")


Created agent with model: gpt-4o
System prompt: You are a test agent.


In [ ]:
# | export
@patch
def run_with_tools(self:Agent, prompt:str, max_steps:int=5, **kwargs) -> str:
    """Execute a conversation turn with automatic tool usage.
    
    Uses cosette's toolloop to allow the model to:
    1. Analyze the prompt
    2. Choose appropriate tools
    3. Call tools with proper parameters
    4. Use results to form response
    
    Args:
        prompt: User's input message
        max_steps: Maximum number of tool calls (default: 5)
        **kwargs: Additional arguments passed to toolloop
    
    Returns:
        The model's final response after tool usage
    
    Example:
        ```python
        agent = Agent(role="math tutor", model="gpt-4o", tools=[solve_math])
        response = agent.run_with_tools("What is sqrt(16) + sin(pi/2)?")
        ```
    """
    self._prune_history()
    response = self.chat.toolloop(prompt, max_steps=max_steps, **kwargs)
    return contents(response)

In [ ]:
# | export
@patch
def show(self:Agent):
    """Display agent configuration and conversation history.
    
    Shows:
    - Current role and model
    - System prompt
    - Available tools
    - Token usage statistics
    - Full conversation history
    """
    config_md = f"""
# Agent Configuration

**Role**: {self.role}  
**Model**: {self.model}  
**System**: {self.chat.sp}  
**Memory Size**: {self.memory_size}  
**Tools**: {len(self.tools)} - {', '.join(t.__name__ for t in self.tools)}

## Token Usage
{self.chat.use}

## Conversation History
{self._format_history()}
"""
    display(Markdown(config_md))



In [ ]:
# | export
@patch
def _format_history(self:Agent):
    """Format conversation history for markdown display."""
    lines = []
    if hasattr(self.chat, 'h') and self.chat.h:
        for msg in self.chat.h:
            role = msg.role.capitalize()
            content = msg.content or ""
            lines.append(f"**{role}:** {content}")
    return "\n\n".join(lines)

In [ ]:
# | export
@patch
def _prune_history(self:Agent):
    """Maintain conversation history within memory_size limit."""
    if self.memory_size is None or self.memory_size <= 0: return
    if hasattr(self.chat, 'h') and len(self.chat.h) > (self.memory_size + 1):
        system_msgs = [msg for msg in self.chat.h if msg.role == 'system']
        other_msgs = [msg for msg in self.chat.h if msg.role != 'system'][-self.memory_size:]
        self.chat.h = system_msgs + other_msgs

In [ ]:
# | hide
# Test the methods
test_agent = Agent(
    role="math tutor",
    model=models[2],
    tools=[solve_math],
    system="You are a helpful math tutor. Use tools to verify calculations."
)

# Test run_with_tools
print("Testing run_with_tools:")
response = test_agent.run_with_tools("What is sqrt(16) + sin(pi/2)?")
print(f"Response: {response}\n")

# Show agent state
print("Testing show:")
test_agent.show()

Testing run_with_tools:
Response: The result of \(\sqrt{16} + \sin(\pi/2)\) is \(4 + 1 = 5\).

Testing show:



# Agent Configuration

**Role**: math tutor  
**Model**: gpt-4o  
**System**: You are a helpful math tutor. Use tools to verify calculations.  
**Memory Size**: 10  
**Tools**: 1 - solve_math

## Token Usage
CompletionUsage(completion_tokens=83, prompt_tokens=545, total_tokens=628, completion_tokens_details=None, prompt_tokens_details=None)

## Conversation History
**User:** What is sqrt(16) + sin(pi/2)?

**Assistant:** 

**Tool:** 4.0

**Tool:** 1.0

**Assistant:** The result of \(\sqrt{16} + \sin(\pi/2)\) is \(4 + 1 = 5\).


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()